# Library Installation

In [8]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install pandas-datareader
!pip install yahoo-fin
!pip install requests


In [9]:
import numpy as np
import pandas as pd
import time
from pandas_datareader import data
import matplotlib.pyplot as plt
import yahoo_fin.stock_info as yf #https://theautomatic.net/yahoo_fin-documentation/
import yfinance as yf2

In [10]:
ticker_names=[]
balance_sheet = []
income_statement = []
cfs = []
years = []

In [11]:
def get_data(ticker):
    global ticker_names
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet=yf.get_balance_sheet(ticker)
    income_statement=yf.get_income_statement(ticker)
    cfs=yf.get_cash_flow(ticker)
    years=balance_sheet.columns

## Testing for individual stock

In [26]:
get_data('C09.SI')

In [27]:
balance_sheet

endDate,2020-12-31,2019-12-31,2018-12-31,2017-12-31
Breakdown,,,,
intangibleAssets,1556000,1416000,1643000,2388000
totalLiab,14434335000,11933701000,8611752000,7718168000
totalStockholderEquity,8171990000,10190032000,9710479000,9061064000
minorityInterest,740249000,746306000,2233243000,2254844000
otherCurrentLiab,1427356000,817503000,899341000,1136665000
totalAssets,23676792000,23200257000,20885692000,19364294000
commonStock,1661179000,1661179000,1661179000,1661179000
otherCurrentAssets,85648000,244520000,25009000,74474000
retainedEarnings,6276295000,8337629000,7966274000,7271048000


# Piotroski-F-Score

In [14]:
balance_sheet = []
income_statement = []
cfs = []
years = []
profitability_score=0
leverage_score=0
operating_efficiency_score=0
pe_ratio=0

# Fetch fin statements

In [15]:
def get_data(ticker):    
    global balance_sheet
    global income_statement
    global cfs
    global years    
    balance_sheet=yf.get_balance_sheet(ticker)
    income_statement=yf.get_income_statement(ticker)
    cfs=yf.get_cash_flow(ticker)
    years=balance_sheet.columns

In [16]:
def get_name(ticker):
    global ticker_names
    ticker_names= yf2.Ticker(ticker)
    ticker_names=ticker_names.info['longName']

# PE Ratios

In [17]:
def pe(ticker):
    global pe_ratio
    pe_ratio=yf.get_quote_table(ticker)['PE Ratio (TTM)']
    if pe_ratio != pe_ratio: #check if NaN
        pe_ratio=0

## Defining Profitability

In [18]:
def profitability():
    global profitability_score
    #Criteria 1
    # Scores 1 and 2 - Net income
    net_income=income_statement[years[0]]['netIncome']
    net_income_py=income_statement[years[1]]['netIncome']
    ni_score=1 if net_income > 0 else 0  
    ni_score_2=1 if net_income > net_income_py else 0
    
    #Criteria 2
    # Score 3 -operating cash flow
    op_cf = cfs[years[0]]['totalCashFromOperatingActivities']
    op_cf_score=1 if op_cf>0 else 0
    
    #Criteria 3
    #Score 4-Change in RoA
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets']+balance_sheet[years[2]]['totalAssets'])/2
    RoA=net_income/avg_assets
    RoA_py=net_income_py/avg_assets_py
    RoA_score=1 if RoA > RoA_py else 0
    
    #Criteria 4
    #Score 5-Accruals
    total_assets=balance_sheet[years[0]]['totalAssets']
    accurals=op_cf/total_assets - RoA
    ac_score = 1 if accurals>0 else 0
    
    profitability_score=ni_score+ni_score_2+op_cf_score+RoA_score+ac_score
    print("Profitability Score: "+str(profitability_score))

## Defining Leverage

In [19]:
def leverage():
    global leverage_score
    #Criteria 5 (Modified long-term levarge ratio)
    #Score 6 - Long-term debt ratio
    try:
        lt_debt=balance_sheet[years[0]]['longTermDebt']
        total_assets=balance_sheet[years[0]]['totalAssets']
        debt_ratio=lt_debt/total_assets        
        debt_ratio_score=1 if debt_ratio<0.4 else 0        
    except:
        debt_ratio_score=1
    
        
    #Criteria 6 (Modified current ratio)
    #Score 7 -Current ratio
    current_assets=balance_sheet[years[0]]['totalCurrentAssets']
    current_liab=balance_sheet[years[0]]['totalCurrentLiabilities']
    current_ratio=current_assets/current_liab   
    current_ratio_score=1 if current_ratio>1 else 0
    
    leverage_score = debt_ratio_score + current_ratio_score
    print("Leverage Score: "+str(leverage_score))

## Defining Operating Efficiency

In [20]:
def operating_efficiency():
    global operating_efficiency_score
    #Criteria 8
    #Score 8-Gross margin
    gp=income_statement[years[0]]['grossProfit']
    gp_py=income_statement[years[1]]['grossProfit']
    revenue=income_statement[years[0]]['totalRevenue']
    revenue_py=income_statement[years[1]]['totalRevenue']
    gm=gp/revenue
    gm_py=gp_py/revenue_py
    gm_score=1 if gm>gm_py else 0
    
    #Criteria 9
    #Score 9-Asset Turnover
    avg_assets=(balance_sheet[years[0]]['totalAssets']+balance_sheet[years[1]]['totalAssets'])/2
    avg_assets_py=(balance_sheet[years[1]]['totalAssets']+balance_sheet[years[2]]['totalAssets'])/2
    asset_turnover=revenue/avg_assets  
    asset_turnover_py=revenue_py/avg_assets_py
    asset_turnover_score=1 if asset_turnover>asset_turnover_py else 0
    
    operating_efficiency_score=gm_score+asset_turnover_score
    print("Operating Efficiency: "+str(operating_efficiency_score))

In [21]:
tickers=['CLN.SI','A30.SI','B61.SI','BTE.SI','OU8.SI','C29.SI','C33.SI','C09.SI','ADN.SI','TQ5.SI','586.SI','5PC.SI','F17.SI','5DP.SI'
        ,'5JK.SI','H13.SI','H30.SI','H78.SI','H19.SI','41O.SI','LJ3.SI','5UX.SI','OYY.SI','E8Z.SI','A26.SI','5IC.SI','U06.SI','5H0.SI'
        ,'AWI.SI','T24.SI','CHJ.SI','EH5.SI','U14.SI','W05.SI','Z25.SI','Z59.SI']
summary=pd.DataFrame(columns = ['Ticker Symbol', 'Ticker Name', 'PE ratio', 'Profitability', 'Leverage', 'Operating efficiency'])
for ticker in tickers:
    try:
        get_data(ticker)
        get_name(ticker)
        pe(ticker)
        print(ticker)
        profitability()
        leverage()
        operating_efficiency()
        new_row={'Ticker Symbol':ticker,
                 'Ticker Name': ticker_names,
                 'PE ratio':pe_ratio,
                 'Profitability':profitability_score,
                 'Leverage':leverage_score,
                 'Operating efficiency':operating_efficiency_score}
        summary=summary.append(new_row, ignore_index=True)
        print(ticker+' added.')    
        time.sleep(3)
    except:
        print(ticker+':Something went wrong.')

summary['Total score'] =summary['Profitability']+summary['Leverage']+summary['Operating efficiency']
summary.to_csv('Summary.csv')   

CLN.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 0
CLN.SI added.
A30.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 0
A30.SI added.
B61.SI
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 1
B61.SI added.
BTE.SI
Profitability Score: 3
Leverage Score: 2
Operating Efficiency: 0
BTE.SI added.
OU8.SI
Profitability Score: 3
Leverage Score: 0
Operating Efficiency: 0
OU8.SI added.
C29.SI
Profitability Score: 2
Leverage Score: 1
Operating Efficiency: 0
C29.SI added.
C33.SI
Profitability Score: 4
Leverage Score: 2
Operating Efficiency: 0
C33.SI added.
C09.SI
Profitability Score: 1
Leverage Score: 2
Operating Efficiency: 0
C09.SI added.
ADN.SI
Profitability Score: 3
Leverage Score: 2
Operating Efficiency: 1
ADN.SI added.
TQ5.SI
Profitability Score: 2
Leverage Score: 1
Operating Efficiency: 1
TQ5.SI added.
586.SI
Profitability Score: 1
Leverage Score: 2
Operating Efficiency: 0
586.SI added.
5PC.SI
Profitability Score: 2
Leverage Score: 2
Operat

In [22]:
summary

,Ticker Symbol,Ticker Name,PE ratio,Profitability,Leverage,Operating efficiency,Total score
0,CLN.SI,APAC Realty Limited,11.60,5,2,0,7
1,A30.SI,Aspial Corporation Limited,19.17,5,2,0,7
2,B61.SI,Bukit Sembawang Estates Limited,7.87,5,2,1,8
3,BTE.SI,Bund Center Investment Ltd,44.62,3,2,0,5
4,OU8.SI,Centurion Corporation Limited,55.00,3,0,0,3
5,C29.SI,Chip Eng Seng Corporation Ltd,0.00,2,1,0,3
6,C33.SI,Chuan Hup Holdings Limited,12.11,4,2,0,6
7,C09.SI,City Developments Limited,0.00,1,2,0,3
8,ADN.SI,First Sponsor Group Limited,15.06,3,2,1,6
9,TQ5.SI,Frasers Property Limited,5.27,2,1,1,4


In [31]:
summary=summary.sort_values(by=['Total score'], ascending=False)

# US Stocks Test

In [24]:
test_summary=pd.DataFrame(columns = ['Ticker', 'Ticker Name', 'PE ratio', 'Profitability', 'Leverage', 'Operating efficiency'])
test_tickers=['msft','aapl','googl','nio']
for ticker in test_tickers:
    try:
        get_data(ticker)
        pe(ticker)
        print(ticker)
        get_name(ticker)
        profitability()
        leverage()
        operating_efficiency()
        new_row={'Ticker':ticker,
                 'Ticker Name':ticker_names,
                 'PE ratio':pe_ratio,
                 'Profitability':profitability_score,
                 'Leverage':leverage_score,
                 'Operating efficiency':operating_efficiency_score}
        test_summary=test_summary.append(new_row, ignore_index=True)
        print(ticker+' added.')    
#         time.sleep(3)
    except:
        print(ticker+':Something went wrong.')

test_summary['Total score'] =test_summary['Profitability']+test_summary['Leverage']+test_summary['Operating efficiency']

msft
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 2
msft added.
aapl
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 2
aapl added.
googl
Profitability Score: 5
Leverage Score: 2
Operating Efficiency: 0
googl added.
nio
Profitability Score: 4
Leverage Score: 2
Operating Efficiency: 2
nio added.


In [25]:
test_summary

,Ticker,Ticker Name,PE ratio,Profitability,Leverage,Operating efficiency,Total score
0,msft,Microsoft Corporation,45.63,5,2,2,9
1,aapl,Apple Inc.,33.72,5,2,2,9
2,googl,Alphabet Inc.,39.57,5,2,0,7
3,nio,NIO Inc.,0.00,4,2,2,8


In [30]:
pe_ratio=yf.get_quote_table('B61.SI')['PE Ratio (TTM)']
pe_ratio

7.87

In [36]:
summary.loc[:, summary.columns!='PE ratio']

,Ticker Symbol,Ticker Name,Profitability,Leverage,Operating efficiency,Total score
22,OYY.SI,PropNex Limited,5,2,2,9
31,EH5.SI,United Overseas Australia Limited,5,2,1,8
2,B61.SI,Bukit Sembawang Estates Limited,5,2,1,8
21,5UX.SI,Oxley Holdings Limited,5,2,1,8
19,41O.SI,LHN Limited,5,2,1,8
33,W05.SI,Wing Tai Holdings Limited,5,2,1,8
29,T24.SI,Tuan Sing Holdings Limited,5,2,1,8
1,A30.SI,Aspial Corporation Limited,5,2,0,7
0,CLN.SI,APAC Realty Limited,5,2,0,7
8,ADN.SI,First Sponsor Group Limited,3,2,1,6
